<a href="https://colab.research.google.com/github/thaynaribeiro/pythonfunctions/blob/main/web_scraping_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.5 MB/s eta 0:00:00


In [15]:
import os
import pandas as pd
import unidecode
from bs4 import BeautifulSoup

In [8]:
#Files Path
election_path = '/content/drive/MyDrive/desenvolvimentos python/análise eleitoral municipal 2024/'

###Load files

In [9]:
neighbourhood_zones_html = os.path.join(election_path,'zonas-eleitorais-bairros.html')

### Building neighbourhoods from html file

In [10]:
# Load HTML file
with open(neighbourhood_zones_html, "r", encoding="utf-8") as file:
    html_content = file.read()

# HTML parsing with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

In [16]:
# Function to extract the electoral zone, address, and neighborhood
def extract_information(soup):
    # Find all occurrences of 'Zona Eleitoral' and 'Endereço'
    zonas = soup.find_all('p', text=lambda x: x and 'Zona Eleitoral' in x)
    enderecos = soup.find_all('p', text=lambda x: x and 'Endereço' in x)

    # Initializing lists to store the data
    zone_list = []
    address_list = []
    neighborhood_list = []

    # Iterate over all occurrences of zones and addresses
    for zona_tag, endereco_tag in zip(zonas, enderecos):
        # Extract the Electoral Zone
        zone = zona_tag.get_text(strip=True)
        zone = zone.split('Zona Eleitoral:')[-1].strip()
        zone = zone.replace('<span>&nbsp;</span></strong>',"")
        zone = zone.replace('</p>',"")

        # Extract the Address
        address = endereco_tag.get_text(strip=True)
        address = address.split('Endereço:')[-1].strip()  # Taking the value after 'Endereço:'
        address = address.replace('<span>&nbsp;</span></strong>',"")

        # Extract the neighborhood (between the last comma and ' - RIO DE JANEIRO')
        neighborhood = address.split(',')[-1].split(' - ')[0].strip()

        # Add the extracted values to the lists
        zone_list.append(zone)
        address_list.append(address)
        neighborhood_list.append(neighborhood)

    # Create a DataFrame with the data
    neighborhoods_zones = pd.DataFrame({
        'ZONA': zone_list,
        'ENDERECO': address_list,
        'BAIRRO': neighborhood_list
    })

    neighborhoods_zones['ZONA'] = neighborhoods_zones['ZONA'].astype(str)
    neighborhoods_zones['ENDERECO'] = neighborhoods_zones['ENDERECO'].astype(str)
    neighborhoods_zones['BAIRRO'] = neighborhoods_zones['BAIRRO'].astype(str)
    neighborhoods_zones['BAIRRO'] = neighborhoods_zones['BAIRRO'].apply(lambda x: unidecode.unidecode(x).upper())

    return neighborhoods_zones

# Extracting information from the HTML
neighborhoods_zones = extract_information(soup)

# Display the resulting DataFrame
neighborhoods_zones.head()

<ipython-input-16-b28d1ab2e66c>:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  zonas = soup.find_all('p', text=lambda x: x and 'Zona Eleitoral' in x)
<ipython-input-16-b28d1ab2e66c>:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  enderecos = soup.find_all('p', text=lambda x: x and 'Endereço' in x)


,ZONA,ENDERECO,BAIRRO
0,4,"RUA JARDIM BOTÂNICO, 1060, JARDIM BOTÂNICO - R...",JARDIM BOTANICO
1,5,"RUA MIGUEL LEMOS, NÚMERO 97, COPACABANA - RIO ...",COPACABANA
2,7,"RUA ANTÔNIO BASÍLIO, 76, TIJUCA - RIO DE JANEI...",TIJUCA
3,8,"RUA CACHAMBI, 328, CACHAMBI - RIO DE JANEIRO</p>",CACHAMBI
4,9,"AV AYRTON SENNA 2001 BLOCO C, BARRA DA TIJUCA ...",BARRA DA TIJUCA
